In [44]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

from tqdm import tqdm
%matplotlib inline

In [45]:
from molvs.charge import Uncharger
from molvs.standardize import Standardizer
from molvs.validate import Validator

from rdkit import Chem

uncharger = Uncharger()
standarizer = Standardizer()
validate = Validator()

def standarize(SMILES):
    try:
        
        if '.' in SMILES:
            return None
        
        mol = Chem.MolFromSmiles(SMILES)
        mol = uncharger.uncharge(mol)
        mol = standarizer.standardize(mol)
        
        elems = {a.GetSymbol() for a in mol.GetAtoms()}
        if 'C' not in elems:
            return None
        
        for atom in mol.GetAtoms():
            assert atom.GetFormalCharge() == 0

        return Chem.MolToSmiles(mol)
        
    except Exception as ex:
        return None

In [46]:
drug_data = pd.read_excel('drug100.xlsx')
drug_data['valid_smiles'] = drug_data.smile.apply(standarize)
valid_drugs = drug_data.dropna(subset=['valid_smiles', 'lowest BDE radica1', 'lowest BDE radical 2'])
valid_drugs.head()

,name,smile,resouces,lowest BDE radica1,lowest BDE radical 2,valid_smiles
0,carbamazepine,C1=CC=C2C(=C1)C=CC3=CC=CC=C3N2C(=O)N,https://pdf.sciencedirectassets.com/271932/1-s...,O=C(N)N1C2=CC=CC=C2[C]=CC3=CC=CC=C31,[H],NC(=O)N1c2ccccc2C=Cc2ccccc21
1,zaleplon,CCN(C1=CC=CC(=C1)C2=CC=NC3=C(C=NN23)C#N)C(=O)C,https://pdf.sciencedirectassets.com/271932/1-s...,C[CH]N(C(C)=O)C1=CC=CC(C2=CC=NC3=C(C#N)C=NN23)=C1,[H],CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1
2,cortisol,CC12CCC(=O)C=C1CCC3C2C(CC4(C3CCC4(C(=O)CO)O)C)O,https://pdf.sciencedirectassets.com/271932/1-s...,CC12CCC(C=C1[CH]CC3C2C(O)CC4(C)C3CCC4(O)C(CO)=...,[H],CC12CCC(=O)C=C1CCC1C2C(O)CC2(C)C1CCC2(O)C(=O)CO
4,lovastatin,CC[C@@H](C(O[C@H]1C[C@@H](C)C=C2C1C(CC[C@H]3C[...,https://pdf.sciencedirectassets.com/271932/1-s...,CC[C@@H](C(O[C@H]1C[C](C)C=C2C1C(CC[C@H]3C[C@H...,[H],CC[C@H](C)C(=O)O[C@H]1C[C@@H](C)C=C2C=CC(C)C(C...
5,quinidine,COC1=CC2=C([C@H](O)C3CC4CCN3C[C@@H]4C=C)C=CN=C...,https://pdf.sciencedirectassets.com/271932/1-s...,COC1=CC2=C([C@H](O)C3CC4CCN3C[C]4C=C)C=CN=C2C=C1,[H],C=C[C@H]1CN2CCC1CC2[C@@H](O)c1ccnc2ccc(OC)cc12


In [47]:
from bde.fragment import canonicalize_smiles, fragment_iterator

In [48]:
ml_bdes = pd.read_csv('final_2D.csv', index_col=0)
dft_bdes = pd.read_csv('drug100.csv', index_col=0)
ml_bdes.head()

,smiles,bond_index,ref,preprocessor_class,mol_id,predicted
0,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,24,111.45,6.0,0,111.253006
1,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,31,110.93,6.0,0,110.419340
2,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,26,110.93,6.0,0,110.419340
3,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,25,111.23,6.0,0,111.233640
4,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,23,111.23,6.0,0,111.233640


In [49]:
dft_bdes.head()

,molecule,radical1,radical2,bond_index,BDE,mol_id
0,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,O=C1NC(=O)C(c2cc[c]cc2)(c2ccccc2)N1,[H],29,111.45,1008
1,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,O=C1NC(=O)C(c2[c]cccc2)(c2ccccc2)N1,[H],31,110.93,1008
3,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,O=C1NC(=O)C(c2c[c]ccc2)(c2ccccc2)N1,[H],23,111.23,1008
10,Cc1c([N+](=O)[O-])cccc1[N+](=O)[O-],Cc1c([N+](=O)[O-])c[c]cc1[N+](=O)[O-],[H],17,113.17,1009
11,Cc1c([N+](=O)[O-])cccc1[N+](=O)[O-],[CH2]c1c([N+](=O)[O-])cccc1[N+](=O)[O-],[H],14,NaN,1009


In [50]:
def drug_fragment_iterator():
    for smiles in tqdm(valid_drugs.valid_smiles):
        for series in fragment_iterator(smiles):
            yield series
            
drug_rdf = pd.DataFrame(drug_fragment_iterator())

100%|██████████| 28/28 [00:03<00:00,  9.83it/s]


In [64]:
drug_rdf_dedupe = drug_rdf[drug_rdf.bond_type == 'C-H'].drop_duplicates(
    ['molecule', 'fragment1', 'fragment2'])

In [67]:
dft_bdes

,molecule,radical1,radical2,bond_index,BDE,mol_id
0,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,O=C1NC(=O)C(c2cc[c]cc2)(c2ccccc2)N1,[H],29,111.45,1008
1,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,O=C1NC(=O)C(c2[c]cccc2)(c2ccccc2)N1,[H],31,110.93,1008
3,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,O=C1NC(=O)C(c2c[c]ccc2)(c2ccccc2)N1,[H],23,111.23,1008
10,Cc1c([N+](=O)[O-])cccc1[N+](=O)[O-],Cc1c([N+](=O)[O-])c[c]cc1[N+](=O)[O-],[H],17,113.17,1009
11,Cc1c([N+](=O)[O-])cccc1[N+](=O)[O-],[CH2]c1c([N+](=O)[O-])cccc1[N+](=O)[O-],[H],14,NaN,1009
13,Cc1c([N+](=O)[O-])cccc1[N+](=O)[O-],Cc1c([N+](=O)[O-])[c]ccc1[N+](=O)[O-],[H],18,113.79,1009
16,COc1ccccc1OCC(O)CO,[CH2]Oc1ccccc1OCC(O)CO,[H],15,97.05,1010
17,COc1ccccc1OCC(O)CO,COc1ccccc1OCC(O)[CH]O,[H],25,93.82,1010
19,COc1ccccc1OCC(O)CO,COc1[c]cccc1OCC(O)CO,[H],17,110.81,1010
20,COc1ccccc1OCC(O)CO,COc1ccccc1O[CH]C(O)CO,[H],21,95.56,1010


In [53]:
dft_bdes.shape

(1007, 6)

In [57]:
dft_bdes

,molecule,radical1,radical2,bond_index,BDE,mol_id
0,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,O=C1NC(=O)C(c2cc[c]cc2)(c2ccccc2)N1,[H],29,111.45,1008
1,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,O=C1NC(=O)C(c2[c]cccc2)(c2ccccc2)N1,[H],31,110.93,1008
3,O=C1NC(=O)C(c2ccccc2)(c2ccccc2)N1,O=C1NC(=O)C(c2c[c]ccc2)(c2ccccc2)N1,[H],23,111.23,1008
10,Cc1c([N+](=O)[O-])cccc1[N+](=O)[O-],Cc1c([N+](=O)[O-])c[c]cc1[N+](=O)[O-],[H],17,113.17,1009
11,Cc1c([N+](=O)[O-])cccc1[N+](=O)[O-],[CH2]c1c([N+](=O)[O-])cccc1[N+](=O)[O-],[H],14,NaN,1009
13,Cc1c([N+](=O)[O-])cccc1[N+](=O)[O-],Cc1c([N+](=O)[O-])[c]ccc1[N+](=O)[O-],[H],18,113.79,1009
16,COc1ccccc1OCC(O)CO,[CH2]Oc1ccccc1OCC(O)CO,[H],15,97.05,1010
17,COc1ccccc1OCC(O)CO,COc1ccccc1OCC(O)[CH]O,[H],25,93.82,1010
19,COc1ccccc1OCC(O)CO,COc1[c]cccc1OCC(O)CO,[H],17,110.81,1010
20,COc1ccccc1OCC(O)CO,COc1ccccc1O[CH]C(O)CO,[H],21,95.56,1010


In [72]:
dft_bdes[dft_bdes.molecule == 'CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1']

,molecule,radical1,radical2,bond_index,BDE,mol_id
107,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,CCN(C(C)=O)c1c[c]cc(-c2ccnc3c(C#N)cnn23)c1,[H],34,NaN,1018
108,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,[CH2]CN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,[H],25,NaN,1018
109,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,CCN(C(C)=O)c1cccc(-c2[c]cnc3c(C#N)cnn23)c1,[H],36,NaN,1018
110,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)[c]nn23)c1,[H],38,NaN,1018
111,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,[CH2]C(=O)N(CC)c1cccc(-c2ccnc3c(C#N)cnn23)c1,[H],31,NaN,1018
112,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,CCN(C(C)=O)c1[c]ccc(-c2ccnc3c(C#N)cnn23)c1,[H],33,NaN,1018
115,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,C[CH]N(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,[H],29,NaN,1018
116,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,CCN(C(C)=O)c1[c]c(-c2ccnc3c(C#N)cnn23)ccc1,[H],39,NaN,1018
117,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,CCN(C(C)=O)c1cccc(-c2c[c]nc3c(C#N)cnn23)c1,[H],37,NaN,1018
121,CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1,CCN(C(C)=O)c1cc[c]c(-c2ccnc3c(C#N)cnn23)c1,[H],35,NaN,1018


In [136]:
drug_rdf_dedupe.shape

(268, 7)

In [137]:
drug_rdf_dft = drug_rdf_dedupe.merge(dft_bdes[['molecule', 'radical1', 'BDE']],
                                     how='left', left_on=['molecule', 'fragment2'],
                                     right_on=['molecule', 'radical1']).drop(['radical1'], 1)

In [138]:
drug_rdf_dft.shape

(268, 8)

In [139]:
drug_ml_predictions = drug_rdf.merge(ml_bdes[['smiles', 'bond_index', 'predicted']],
                                     left_on=['molecule', 'bond_index'],
                                     right_on=['smiles', 'bond_index'],
                                     how='left').dropna(subset=['predicted']).drop_duplicates(
    ['molecule', 'fragment1', 'fragment2'])

In [140]:
drug_rdf_dft_ml = drug_rdf_dft.merge(drug_ml_predictions[['molecule', 'fragment2', 'predicted']],
                                     on=['molecule', 'fragment2'], how='left')

In [141]:
drug_rdf_dft_ml.shape

(268, 9)

In [147]:
drug_rdf_dft_ml.groupby('molecule').BDE.apply(lambda x: sum(x.isna()))

molecule
C=C(C)[C@H]1CC=C(C)CC1                                                         1
C=C[C@H]1CN2CCC1CC2[C@@H](O)c1ccnc2ccc(OC)cc12                                 2
CC(=O)Oc1ccccc1C(=O)O                                                          1
CC(=O)[C@]1(C)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@]4(C)[C@H]3CC[C@@]21C             4
CC(C)(C)C1CCC(C2=C(O)C(=O)c3ccccc3C2=O)CC1                                     1
CC12CCC(=O)C=C1CCC1C2C(O)CC2(C)C1CCC2(O)C(=O)CO                                5
CC1=NC2C(C(O)=CC(=O)N2Cc2ccc(-c3ccccc3-c3nnn[nH]3)cc2)C(C)=N1                  4
CCCC(CCC)C(=O)O                                                                0
CCCCCOc1ccc2ccc(=O)oc2c1                                                       1
CCCCCc1cc(O)c2c(c1)OC(C)(C)[C@@H]1CCC(C)=C[C@@H]21                             2
CCN(C(C)=O)c1cccc(-c2ccnc3c(C#N)cnn23)c1                                      10
CCN(CC)CC(=O)Nc1c(C)cccc1C                                                     0
CCOC(=O)C1=C(COCCN)

In [148]:
drug_rdf_dft_ml.shape

(268, 9)

In [149]:
drug_rdf_dft_ml.BDE.isna().sum()

52

In [157]:
.isin(drug_rdf_dft_ml.fragment2)

0     True
1     True
2     True
4     True
5     True
6     True
7     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
20    True
21    True
22    True
23    True
24    True
25    True
26    True
27    True
28    True
29    True
Name: lowest BDE radica1, dtype: bool

In [160]:
drug_rdf_dft_ml['is_metab_site'] = drug_rdf_dft_ml.fragment2.isin(
    valid_drugs['lowest BDE radica1'].apply(canonicalize_smiles))

In [178]:
drug_rdf_dft_ml_with_molid = drug_rdf_dft_ml.merge(
    dft_bdes.drop_duplicates(subset=['molecule', 'mol_id'])[['molecule', 'mol_id']],
    how='left', on=['molecule'])

In [179]:
drug_rdf_dft_ml_with_molid.to_csv('drug_bdes.csv')

In [143]:
dft_bdes[dft_bdes.molecule == 'C=C(C)[C@H]1CC=C(C)CC1']

,molecule,radical1,radical2,bond_index,BDE,mol_id
608,C=C(C)[C@H]1CC=C(C)CC1,[CH]=C(C)[C@H]1CC=C(C)CC1,[H],11,109.43,1049
609,C=C(C)[C@H]1CC=C(C)CC1,[CH2]C1=CC[C@H](C(=C)C)CC1,[H],21,NaN,1049
611,C=C(C)[C@H]1CC=C(C)CC1,C=C(C)[C@H]1[CH]C=C(C)CC1,[H],16,84.07,1049
612,C=C(C)[C@H]1CC=C(C)CC1,[CH2]C(=C)[C@H]1CC=C(C)CC1,[H],13,88.16,1049
614,C=C(C)[C@H]1CC=C(C)CC1,C=C(C)[C@@H]1C[CH]C(C)=CC1,[H],22,85.23,1049
616,C=C(C)[C@H]1CC=C(C)CC1,C=C(C)[C@@H]1[CH]CC(C)=CC1,[H],25,97.67,1049
618,C=C(C)[C@H]1CC=C(C)CC1,C=C(C)[C]1CC=C(C)CC1,[H],15,85.04,1049
621,C=C(C)[C@H]1CC=C(C)CC1,C=C(C)[C@H]1C[C]=C(C)CC1,[H],18,108.04,1049
